In [4]:
from datetime import datetime, timezone, timedelta
startDate = datetime(2011, 1, 23)
endDate = datetime(2012, 1, 23, 4, 30, 0)

In [7]:
import cdflib
from pathlib import Path

windPosition_km = pd.DataFrame(data=None, index=None)
windVelocity_km_s = pd.DataFrame(data=None, index=None)
windDateTime = pd.Series(data=None)

p = Path('.')
for windFileName in sorted(p.glob('wind_trajectory/*/*.cdf')):
    # Browsing through daily data files in the folders
    fileNameWithoutPath = windFileName.name
    fileYear = int(fileNameWithoutPath[10:14])
    fileMonth = int(fileNameWithoutPath[14:16])
    fileDay = int(fileNameWithoutPath[16:18])
    fileDate = datetime(fileYear, fileMonth, fileDay)
    if (fileDate >= startDate) and (fileDate < endDate):
        cdfFile = cdflib.CDF(windFileName)
        # Getting position in Geocentric solar ecliptic (GSE) 
        # Other frames are available in the CDF file:
        #   - Geocentric equatorial inertial (GCI) frame
        #   - Geocentric solar magnetic (GSM)
        timePb5 = cdfFile.varget(variable='Time_PB5')
        currentDateTime = pd.Series([datetime(pb5cur[0], 1, 1) + timedelta(days=pb5cur[1].item()-1) + timedelta(milliseconds=pb5cur[2].item()) for pb5cur in timePb5])
        gsePos = pd.DataFrame(data=cdfFile.varget(variable='GCI_POS'),
                             index=currentDateTime,
                             columns=['x','y','z'])
        gseVel = pd.DataFrame(data=cdfFile.varget(variable='GCI_VEL'),
                             index=currentDateTime,
                             columns=['vx','vy','vz'])
        
        windPosition_km = pd.concat([windPosition_km, gsePos])
        windVelocity_km_s = pd.concat([windVelocity_km_s, gseVel])
        windDateTime = pd.concat([windDateTime, currentDateTime])

Cropping the data to the date range

In [8]:
windPosition_km = windPosition_km.loc[windPosition_km.index <= endDate]

Writing position velocity to csv

In [9]:
wind_pv_km = pd.concat([windPosition_km, windVelocity_km_s], axis=1, join='inner')
wind_pv_km.to_csv('wind-pv-km-eme2000.csv')